In [ ]:
#| hide
from ouscope import *

# OUScope

> The `ouscope` is a library providing python interface to the Open University Autonomous Robotic Telescope facility at telescope.org

The library provides access and control for the telescope. The functions help with job submission, image retrival and initial analysis. For naw this is aimed at variable star observations, but there is nothing prohibiting additional modules intended for different fields (e.g. astrometry).

## Install

`pip install ouscope`

## How to use

Not ready yet so you cannot really use it now.